In [ ]:
session.write_pandas(dim_date, "DIM_DATE", auto_create_table=True, overwrite=True)

# DIM_PATIENT
session.write_pandas(dim_patient, "DIM_PATIENT", auto_create_table=True, overwrite=True)

# DIM_DIAGNOSIS
session.write_pandas(dim_diagnosis, "DIM_DIAGNOSIS", auto_create_table=True, overwrite=True)

# FACT_PATIENT_DIAGNOSIS
session.write_pandas(fact_patient_diagnosis, "FACT_PATIENT_DIAGNOSIS", auto_create_table=True, overwrite=True)

In [ ]:
fact_patient_diagnosis = df_clean[["PATIENT_DIAGNOSIS_SK", "PATIENT_SK", "DATE_SK", "DIAGNOSIS_SK", "AVG_GLUCOSE_LEVEL","BMI"]]


In [ ]:
dim_diagnosis = df_clean[["DIAGNOSIS_SK", "HYPERTENSION", "SMOKING_STATUS", "HEART_DISEASE","STROKE","DIAGNOSIS_DATE"]].drop_duplicates()

In [ ]:
print(df_clean.columns.tolist())

['PATIENT_ID', 'GENDER', 'AGE', 'HYPERTENSION', 'HEART_DISEASE', 'EVER_MARRIED', 'WORK_TYPE', 'RESIDENCE_TYPE', 'AVG_GLUCOSE_LEVEL', 'BMI', 'SMOKING_STATUS', 'STROKE', 'DATE_SK', 'PATIENT_SK', 'DIAGNOSIS_SK', 'PATIENT_DIAGNOSIS_SK', 'DIAGNOSIS_DATE', 'FULL_DATE', 'YEAR', 'MONTH', 'DAY']


In [ ]:
dim_patient = df_clean[["PATIENT_SK", "PATIENT_ID", "GENDER", "AGE",
                        "EVER_MARRIED", "WORK_TYPE",
                        "RESIDENCE_TYPE"]].drop_duplicates()

In [ ]:
dim_date = df_clean[["DATE_SK", "FULL_DATE", "YEAR", "MONTH", "DAY"]].drop_duplicates()

In [ ]:
#EXTRACT YEAR, MONTH, DAY
df_clean["FULL_DATE"] = pd.to_datetime(df_clean["FULL_DATE"])

df_clean["YEAR"] = df_clean["FULL_DATE"].dt.year
df_clean["MONTH"] = df_clean["FULL_DATE"].dt.month
df_clean["DAY"] = df_clean["FULL_DATE"].dt.day

In [ ]:
df_clean["FULL_DATE"] = df_clean["FULL_DATE"].dt.date

In [ ]:
#DATA TYPE DIDN'T ACCEPT ANY CHANGES UNTIL I WROTE THIS CODE
df_clean["FULL_DATE"] = pd.to_datetime(df_clean["FULL_DATE"], unit='ns')

In [ ]:
#ADDING FULL DATE FOR DATE DIM
import pandas as pd

start_date = pd.to_datetime("2022-01-01")
end_date = pd.to_datetime("2025-01-01")

days_range = (end_date - start_date).days

df_clean["FULL_DATE"] = pd.date_range(start=start_date, periods=len(df_clean), freq="D")

In [ ]:
#THERE WAS A PROBLEM AFTER EXPORTIN DATA SO I CHANGED THE TYPE
df_clean["DIAGNOSIS_DATE"] = df_clean["DIAGNOSIS_DATE"].dt.date

In [ ]:
#ASSUMING DATES FOR DIAGNOSIS
import numpy as np

start_date = pd.to_datetime("2022-01-01")
end_date = pd.to_datetime("2025-1-1")

days_range = (end_date - start_date).days

df_clean["DIAGNOSIS_DATE"] = [start_date + pd.Timedelta(days=int(x))
                           for x in np.random.randint(0, days_range, size=len(df_clean))]

In [ ]:
#CHANGE ID NAME TO EASIER KNOWING
df_clean.rename(columns={"ID": "PATIENT_ID"}, inplace=True)

In [ ]:
#ADDING SURRGET KEYS
df_clean["DATE_SK"] = range(1, len(df_clean) + 1)
df_clean["PATIENT_SK"] = range(1, len(df_clean) + 1)
df_clean["DIAGNOSIS_SK"] = range(1, len(df_clean) + 1)
df_clean["PATIENT_DIAGNOSIS_SK"] = range(1, len(df_clean) + 1)

In [78]:
#SAVING CHANGES IN DATA WARE HOUSE
session.write_pandas(df_clean, "ALL_DATA_CLEAN", auto_create_table=True, overwrite=True)

In [ ]:
# check duplicates
df.duplicated().sum()

np.int64(0)

In [ ]:
#1 assuming this hospital not for under 25 and not above 100
import numpy as np

df_clean.loc[(df_clean["AGE"] < 25) | (df_clean["AGE"] > 100), "AGE"] = np.nan

In [ ]:
#convert nulls to -1 / converting type due to the problem happenned
df_clean["BMI"] = pd.to_numeric(df_clean["BMI"], errors="raise")

df_clean["BMI"] = df_clean["BMI"].fillna(-1)
print(df_clean.isnull().sum())

ID                   0
GENDER               0
AGE                  0
HYPERTENSION         0
HEART_DISEASE        0
EVER_MARRIED         0
WORK_TYPE            0
RESIDENCE_TYPE       0
AVG_GLUCOSE_LEVEL    0
BMI                  0
SMOKING_STATUS       0
STROKE               0
dtype: int64


In [ ]:
# convert any missing to null
df_clean = df.replace(["N/A", "NA", "-", "?"], None)
print(df_clean.isnull().sum())

ID                     0
GENDER                 0
AGE                    0
HYPERTENSION           0
HEART_DISEASE          0
EVER_MARRIED           0
WORK_TYPE              0
RESIDENCE_TYPE         0
AVG_GLUCOSE_LEVEL      0
BMI                  201
SMOKING_STATUS         0
STROKE                 0
dtype: int64


In [ ]:
import snowflake.snowpark as snowpark


connection_parameters = {
    "account": "",
    "user": "",
    "password": "",
    "role": "ACCOUNTADMIN",
    "warehouse": "COMPUTE_WH",
    "database": "HEALTH_CARE_DWH",
    "schema": "STAR_SCHEMA"
}

session = snowpark.Session.builder.configs(connection_parameters).create()
print("✅ Connected to Snowflake")

✅ Connected to Snowflake


In [ ]:
!pip install snowflake-snowpark-python pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.8/73.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 3.1.1
    Uninstalling cloudpickle-3.1.1:
      Successfully uninstalled cloudpickle-3.1.1


In [ ]:
import pandas as pd